This notebook is used to randomly select a certain number of buses (among those that have GPS coordinates) and save the corresponding variable names to a JSON file to be used in training a CNN for momentum estimation.

In [ ]:
import os
import sys
import json
import numpy as np
from numpy.random import RandomState, SeedSequence, MT19937
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pyproj import CRS

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
fontsize = 9
lw = 0.75
matplotlib.rc('font', **{'family': 'Arial', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})

#### The coordinate reference systems

In [ ]:
# 3035: Lambert azimuthal equal area
# 3857: spherical Mercator projection
# 4326: world geodetic system 1984
source_coord_ref = 4326
coord_ref = 3857

Load the coordinates of the terminals and convert them to the reference system:

In [ ]:
coords_file = os.path.join('..','V2020_Rete_Sardegna_2021_06_03cr_terminals_coords.json')
coords = json.load(open(coords_file))
terminal_names_with_coords = list(coords.keys())
XY = np.array(list(coords.values()))
# remove the terminals in Corse
idx, = np.where(XY[:,0] != np.max(XY[:,0]))
terminal_names_with_coords = [terminal_names_with_coords[i] for i in idx]
terminal_coords = [Point(long,lat) for lat,long in zip(XY[idx,0],XY[idx,1])]
terminal_gdf = gpd.GeoDataFrame(data={'name': terminal_names_with_coords, 'geometry': terminal_coords})
terminal_gdf.crs = CRS.from_user_input(source_coord_ref)
terminal_gdf = terminal_gdf.to_crs(epsg=coord_ref)

Load the coordinates of the sychronous machines and convert them to the reference system:

In [ ]:
coords_file = os.path.join('..','V2020_Rete_Sardegna_2021_06_03cr_coords.json')
coords = json.load(open(coords_file))
SM_names_with_coords = []
XY = []
for name,data in coords['Sym'].items():
    xy = [data['lat'],data['lon']]
    if xy not in XY:
        XY.append(xy)
        SM_names_with_coords.append(name)
XY = np.array(XY)
idx, = np.where(XY[:,0] != np.max(XY[:,0]))
SM_names_with_coords = [terminal_names_with_coords[i] for i in idx]
SM_coords = [Point(long,lat) for lat,long in zip(XY[idx,0],XY[idx,1])]
# SM_coords = [Point(lon,lat) for lat,lon in zip(XY[:,0],XY[:,1])]
SM_gdf = gpd.GeoDataFrame(data={'name': SM_names_with_coords, 'geometry': SM_coords})
SM_gdf.crs = CRS.from_user_input(source_coord_ref)
SM_gdf = SM_gdf.to_crs(epsg=coord_ref)

Pick `N_buses` terminals that have coordinates:

In [ ]:
seed = 41920
if seed is None:
    with open('/dev/urandom', 'rb') as fid:
        seed = int.from_bytes(fid.read(4), 'little') % 100000
rs = RandomState(MT19937(SeedSequence(seed)))

N_buses = 20
data_dir = os.path.join('..','data','Sardinia','SM_configs_from_data','001')
fname = 'V2020_Rete_Sardegna_2021_06_03cr_AC.npz'
TF_fname = 'V2020_Rete_Sardegna_2021_06_03cr_AC_TF_-6.0_2.0_100.npz'
data = np.load(os.path.join(data_dir,fname), allow_pickle=True)
TF_data = np.load(os.path.join(data_dir,TF_fname), allow_pickle=True)
TF = TF_data['TF']
TF_var_names = TF_data['var_names']
vars_idx = data['vars_idx'].item()
all_var_names = list(vars_idx.keys())

extract_terminal_name = lambda name: name.split('-')[-1].split('.')[0]

var_names = []
terminals_idx = []
terminal_names = []
fun = lambda var_name,terminal_name: \
    terminal_name in var_name and \
    'ElmTerm' in var_name and \
    '__BUS__' in var_name
for i in rs.permutation(len(terminal_names_with_coords)):
    terminal_name = terminal_names_with_coords[i]
    if terminal_name in terminal_names:
        print(f'{terminal_name} already in terminal names')
        continue
    full_terminal_names = list(filter(lambda item: fun(item,terminal_name), all_var_names))
    if len(full_terminal_names) == 1:
        var_name = full_terminal_names[0] + '.ur'
        idx, = np.where(TF_var_names == var_name)
        if len(idx) == 1:
            if np.std(10*np.log10(np.abs(TF[0,:,idx[0]]))) > 1e-3:
                terminals_idx.append(i)
                var_names.append(var_name)
    if len(var_names) == N_buses:
        break
    terminal_names.append(terminal_name)

slack = 'CODCTI0201'
bus_names = [n for n in all_var_names if 'ElmTerm' in n and '__BUS__' in n and slack in n]
for name in bus_names:
    var_names.append(name + '.ur')
    terminal_name = name.split('-')[-1].split('.')[0]
    try:
        terminals_idx.append(terminal_names.index(terminal_name))
    except:
        print('Terminal {} does not have GPS coordinates.'.format(terminal_name))
gen_names = [n for n in all_var_names if 'ElmSym' in n and '__GEN__' in n and slack in n]
for name in gen_names:
    var_names.append(name + '.speed')

outfile = os.path.join('..','config','Sardinia',f'var_names_{seed}.json')
json.dump({'var_names': var_names}, open(outfile,'w'), indent=4)
print(f'Saved variable names to file {outfile}.')

### Map of Sardinia

First define the bounding box:

In [ ]:
limits = {'WS': Point(8, 38.75), 'EN': Point(10, 41)}
bbox = gpd.GeoDataFrame(data=limits.values(),
                        index=pd.Index(data=limits.keys(), name='name'),
                        columns=['geometry'])
bbox.crs = CRS.from_user_input(source_coord_ref)
bbox = bbox.to_crs(epsg=coord_ref)

Then load the geo data of Europe and keep only those coordinates that fall within the bounding box:

In [ ]:
scale = 1 # 1 : 1,000,000
year = 2021
europe_folder = f'geography/ref-nuts-{year}-{scale:02d}m'
N_levels = 4
map_types = 'BN', #'LB' # BN: boundary, LB: label, RG: region
europe = {map_type: {} for map_type in map_types}
for level in range(N_levels):
    for map_type in map_types:
        if map_type == 'LB':
            europe_file = f'{europe_folder}/NUTS_{map_type}_{year}_{coord_ref}_LEVL_{level}.json'
        else:
            europe_file = f'{europe_folder}/NUTS_{map_type}_{scale:02d}M_{year}_{coord_ref}_LEVL_{level}.json'
        tmp = gpd.read_file(europe_file)
        tmp.crs = CRS.from_user_input(coord_ref)
        europe[map_type][level] = tmp.cx[bbox.loc['WS','geometry'].x : bbox.loc['EN','geometry'].x,
                                         bbox.loc['WS','geometry'].y : bbox.loc['EN','geometry'].y]

In [ ]:
ms = 8
width,height = 3.5,2.75
fig,ax = plt.subplots(1, 1, figsize=(width, height))
light_gray = .8 + np.zeros(3)
dark_gray = .2 + np.zeros(3)
europe['BN'][0].plot(ax=ax, lw=1.25, color=dark_gray)
europe['BN'][3].plot(ax=ax, lw=0.5, color=light_gray)
# terminal_gdf.plot(marker='o', ax=ax, markersize=ms, color=light_gray-0.3, label='Terminal')
terminal_gdf.iloc[terminals_idx,:].plot(marker='o', ax=ax, markersize=ms*2, color='tab:red',
                                        label='Selected terminal')
SM_gdf.plot(marker='s', ax=ax, markersize=ms*2, color='k', facecolor='k',
            lw=1.5, label='Synch. generator')
ax.legend(loc='lower left', bbox_to_anchor=(-0.85, 0.5, 0.5, 0.3), fontsize=8, frameon=False)
ax.axis('off')
ax.add_artist(ScaleBar(dx=1, fixed_value=50, fixed_units='km', location='lower right'))
fig.tight_layout()
plt.savefig(f'Sardinia_geo_with_selected_terminals_{seed}.pdf')